# Trees & Binary Search Trees

*Hierarchical data structures and ordered search*


## 🎯 Learning Objectives

- Understand tree terminology (root, leaf, height, depth) and distinguish tree types
- Implement all four tree traversals: inorder, preorder, postorder (recursive + iterative), and level-order (BFS)
- Build a Binary Search Tree with insert, search, delete, and find min/max operations
- Analyze BST performance and understand why balanced trees (AVL) are necessary
- Apply common tree patterns: LCA, path sum, serialization, and trie-based autocomplete

---
## 1. Introduction to Trees


So far, we've studied linear data structures: arrays, linked lists, stacks, and queues. Now we move to **trees**—a fundamentally different structure that organizes data hierarchically. Trees are everywhere: file systems, HTML DOM, organization charts, and decision-making algorithms.

> 📖 **Definition:** A **tree** is a hierarchical data structure consisting of nodes connected by edges. It has a single **root** node at the top, and each node can have zero or more **child** nodes. Unlike graphs, trees have no cycles—there's exactly one path between any two nodes.

> 💡 **Tree Visualization**
>
> ```

                    A          ← Root (level 0)
                  / | \
                 B  C  D       ← Level 1
                / \    |
               E   F   G       ← Level 2
                  /|\
                 H I J         ← Level 3 (H, I, J are leaves)

Key observations:
• A is the root (no parent)
• E, H, I, J, G are leaves (no children)
• B is parent of E and F; F is parent of H, I, J
• Depth of A = 0, depth of F = 2, depth of H = 3
• Height of tree = 3 (longest path from root to leaf)
                
```

### Why Trees?

Trees provide efficient operations that neither arrays nor linked lists can match:

| Operation | Array (sorted) | Linked List | Balanced BST |
| --- | --- | --- | --- |
| Search | O(log n) | O(n) | O(log n) |
| Insert | O(n) | O(1)* | O(log n) |
| Delete | O(n) | O(1)* | O(log n) |
| Find Min/Max | O(1) | O(n) | O(log n) |

** If you already have a reference to the position*

**Listing 6.1 — Basic Tree Node**

In [ ]:
# Generic tree node - can have any number of children
class TreeNode:
    def __init__(self, data):
        self.data = data
        self.children = []  # List of child nodes
    
    def add_child(self, child):
        self.children.append(child)
    
    def __repr__(self):
        return f"TreeNode({self.data})"

# Build a simple tree
#       A
#      /|\
#     B C D
#    /|
#   E F
root = TreeNode('A')
b = TreeNode('B')
c = TreeNode('C')
d = TreeNode('D')
e = TreeNode('E')
f = TreeNode('F')

root.add_child(b)
root.add_child(c)
root.add_child(d)
b.add_child(e)
b.add_child(f)

print(f"Root: {root}")
print(f"Root's children: {root.children}")
print(f"B's children: {b.children}")
print(f"C's children: {c.children} (leaf node)")

***Figure 6.1:** A generic tree node stores data and a list of children. Leaves have empty children lists.*

---
## 2. Tree Terminology


Trees have specific vocabulary that's important to understand:

> 📖 **Key Terms:** **Root:** The topmost node with no parent.

**Parent:** A node that has children.

**Child:** A node directly connected below another node.

**Siblings:** Nodes sharing the same parent.

**Leaf:** A node with no children.

**Internal node:** A node with at least one child.

**Depth:** Distance from root to node (root has depth 0).

**Height:** Longest path from node to any leaf below it.

**Subtree:** A node and all its descendants.

**Listing 6.2 — Computing Tree Properties**

In [ ]:
class TreeNode:
    def __init__(self, data):
        self.data = data
        self.children = []
    
    def add_child(self, child):
        self.children.append(child)

def tree_height(node):
    """Height = longest path to leaf. Leaf height = 0."""
    if not node.children:
        return 0
    return 1 + max(tree_height(child) for child in node.children)

def tree_size(node):
    """Total number of nodes in subtree."""
    return 1 + sum(tree_size(child) for child in node.children)

def count_leaves(node):
    """Count leaf nodes in subtree."""
    if not node.children:
        return 1
    return sum(count_leaves(child) for child in node.children)

def find_depth(root, target, depth=0):
    """Find depth of target node."""
    if root.data == target:
        return depth
    for child in root.children:
        result = find_depth(child, target, depth + 1)
        if result != -1:
            return result
    return -1

# Build tree:  A -> [B -> [E, F], C, D -> [G]]
root = TreeNode('A')
b, c, d = TreeNode('B'), TreeNode('C'), TreeNode('D')
e, f, g = TreeNode('E'), TreeNode('F'), TreeNode('G')
root.add_child(b); root.add_child(c); root.add_child(d)
b.add_child(e); b.add_child(f); d.add_child(g)

print(f"Tree height: {tree_height(root)}")
print(f"Tree size: {tree_size(root)}")
print(f"Leaf count: {count_leaves(root)}")
print(f"Depth of 'A': {find_depth(root, 'A')}")
print(f"Depth of 'F': {find_depth(root, 'F')}")
print(f"Depth of 'G': {find_depth(root, 'G')}")

***Figure 6.2:** Tree properties are naturally computed with recursion—process node, then recurse on children.*

### Types of Trees

- **Binary Tree:** Each node has at most 2 children (left and right)
- **Binary Search Tree (BST):** Binary tree with ordering property
- **Balanced Tree:** Height is O(log n) - AVL, Red-Black trees
- **Complete Binary Tree:** All levels filled except possibly last
- **Full Binary Tree:** Every node has 0 or 2 children
- **N-ary Tree:** Each node can have up to N children

---
## 3. Binary Trees


> 📖 **Definition:** A **binary tree** is a tree where each node has at most two children, typically called **left** and **right**. Binary trees are the foundation for many important data structures including BSTs, heaps, and expression trees.

**Listing 6.3 — Binary Tree Node**

In [ ]:
# Binary tree node - exactly left and right children
class BinaryNode:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
    
    def __repr__(self):
        return f"BinaryNode({self.data})"

# Build a binary tree:
#        1
#       / \
#      2   3
#     / \   \
#    4   5   6

root = BinaryNode(1)
root.left = BinaryNode(2)
root.right = BinaryNode(3)
root.left.left = BinaryNode(4)
root.left.right = BinaryNode(5)
root.right.right = BinaryNode(6)

print(f"Root: {root}")
print(f"Left child: {root.left}")
print(f"Right child: {root.right}")
print(f"Left-Left: {root.left.left}")
print(f"Left-Right: {root.left.right}")
print(f"Right-Right: {root.right.right}")

***Figure 6.3:** Binary tree nodes have explicit left and right pointers instead of a children list.*

### Binary Tree Properties

**Listing 6.4 — Binary Tree Properties**

In [ ]:
class BinaryNode:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def height(node):
    """Height of binary tree."""
    if node is None:
        return -1  # Empty tree has height -1
    return 1 + max(height(node.left), height(node.right))

def size(node):
    """Number of nodes."""
    if node is None:
        return 0
    return 1 + size(node.left) + size(node.right)

def is_full(node):
    """Every node has 0 or 2 children."""
    if node is None:
        return True
    if node.left is None and node.right is None:
        return True  # Leaf
    if node.left and node.right:
        return is_full(node.left) and is_full(node.right)
    return False  # One child only

def max_nodes_at_height(h):
    """Maximum nodes in binary tree of height h."""
    return 2**(h+1) - 1

# Build tree
root = BinaryNode(1)
root.left = BinaryNode(2)
root.right = BinaryNode(3)
root.left.left = BinaryNode(4)
root.left.right = BinaryNode(5)

print(f"Height: {height(root)}")
print(f"Size: {size(root)}")
print(f"Is full binary tree: {is_full(root)}")

print(f"\nMax nodes for heights 0-4:")
for h in range(5):
    print(f"  Height {h}: max {max_nodes_at_height(h)} nodes")

***Figure 6.4:** Binary trees have at most 2^(h+1) - 1 nodes, where h is the height.*

---
## 4. Tree Traversals


**Tree traversal** means visiting every node exactly once in a specific order. There are four main traversal methods:

> 💡 **Traversal Orders**
>
> ```

        1
       / \
      2   3
     / \
    4   5

Preorder  (Root, Left, Right): 1, 2, 4, 5, 3
Inorder   (Left, Root, Right): 4, 2, 5, 1, 3
Postorder (Left, Right, Root): 4, 5, 2, 3, 1
Level-order (BFS):             1, 2, 3, 4, 5
                
```

### Recursive Traversals

**Listing 6.5 — Recursive Traversals**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def preorder(node, result=None):
    """Root -> Left -> Right"""
    if result is None:
        result = []
    if node:
        result.append(node.data)    # Visit root
        preorder(node.left, result)  # Traverse left
        preorder(node.right, result) # Traverse right
    return result

def inorder(node, result=None):
    """Left -> Root -> Right"""
    if result is None:
        result = []
    if node:
        inorder(node.left, result)   # Traverse left
        result.append(node.data)     # Visit root
        inorder(node.right, result)  # Traverse right
    return result

def postorder(node, result=None):
    """Left -> Right -> Root"""
    if result is None:
        result = []
    if node:
        postorder(node.left, result)  # Traverse left
        postorder(node.right, result) # Traverse right
        result.append(node.data)      # Visit root
    return result

# Build tree:  1 -> [2 -> [4, 5], 3]
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)

print(f"Preorder:  {preorder(root)}")
print(f"Inorder:   {inorder(root)}")
print(f"Postorder: {postorder(root)}")

***Figure 6.5:** The three DFS traversals differ only in when they visit the current node.*

### Iterative Traversals with Stack

**Listing 6.6 — Iterative Preorder**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def preorder_iterative(root):
    """Preorder using explicit stack."""
    if not root:
        return []
    
    result = []
    stack = [root]
    
    while stack:
        node = stack.pop()
        result.append(node.data)
        
        # Push right first so left is processed first
        if node.right:
            stack.append(node.right)
        if node.left:
            stack.append(node.left)
    
    return result

def inorder_iterative(root):
    """Inorder using explicit stack."""
    result = []
    stack = []
    current = root
    
    while current or stack:
        # Go to leftmost
        while current:
            stack.append(current)
            current = current.left
        
        # Visit node
        current = stack.pop()
        result.append(current.data)
        
        # Move to right subtree
        current = current.right
    
    return result

# Build tree
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)

print(f"Preorder (iterative): {preorder_iterative(root)}")
print(f"Inorder (iterative):  {inorder_iterative(root)}")

***Figure 6.6:** Iterative traversals use a stack to simulate the call stack of recursion.*

### Level-Order Traversal (BFS)

**Listing 6.7 — Level-Order Traversal**

In [ ]:
from collections import deque

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def level_order(root):
    """BFS traversal - level by level."""
    if not root:
        return []
    
    result = []
    queue = deque([root])
    
    while queue:
        node = queue.popleft()
        result.append(node.data)
        
        if node.left:
            queue.append(node.left)
        if node.right:
            queue.append(node.right)
    
    return result

def level_order_by_level(root):
    """Return list of lists, one per level."""
    if not root:
        return []
    
    result = []
    queue = deque([root])
    
    while queue:
        level_size = len(queue)
        level_values = []
        
        for _ in range(level_size):
            node = queue.popleft()
            level_values.append(node.data)
            
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
        
        result.append(level_values)
    
    return result

# Build tree
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)
root.right.right = Node(6)

print(f"Level order: {level_order(root)}")
print(f"By level: {level_order_by_level(root)}")

***Figure 6.7:** Level-order uses a queue to visit nodes breadth-first, one level at a time.*

### Iterative Postorder Traversal

**Listing 6.8 — Iterative Postorder**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def postorder_iterative(root):
    """
    Postorder using two stacks.
    Stack1 processes nodes, Stack2 stores result in reverse.
    """
    if not root:
        return []
    
    stack1 = [root]
    stack2 = []
    
    while stack1:
        node = stack1.pop()
        stack2.append(node.data)
        
        # Push left first so right is processed first
        if node.left:
            stack1.append(node.left)
        if node.right:
            stack1.append(node.right)
    
    return stack2[::-1]  # Reverse to get postorder

# Build tree
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)

print(f"Postorder (iterative): {postorder_iterative(root)}")

***Figure 6.8:** Iterative postorder uses two stacks—one for processing, one for storing results.*

### Morris Traversal (O(1) Space)

**Listing 6.9 — Morris Inorder Traversal**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def morris_inorder(root):
    """
    O(1) space inorder traversal using Morris threading.
    Temporarily modifies tree structure (restores after).
    """
    result = []
    current = root
    
    while current:
        if current.left is None:
            result.append(current.data)
            current = current.right
        else:
            # Find inorder predecessor
            predecessor = current.left
            while predecessor.right and predecessor.right != current:
                predecessor = predecessor.right
            
            if predecessor.right is None:
                # Create thread
                predecessor.right = current
                current = current.left
            else:
                # Remove thread
                predecessor.right = None
                result.append(current.data)
                current = current.right
    
    return result

# Build tree
root = Node(4)
root.left = Node(2)
root.right = Node(6)
root.left.left = Node(1)
root.left.right = Node(3)
root.right.left = Node(5)

print(f"Morris inorder: {morris_inorder(root)}")
print("Note: O(1) space, O(n) time!")

***Figure 6.9:** Morris traversal uses temporary threading to achieve O(1) space without recursion or stack.*

---
## 5. Binary Search Trees


> 📖 **Definition:** A **Binary Search Tree (BST)** is a binary tree with an ordering property: for every node, all values in its left subtree are less than the node's value, and all values in its right subtree are greater. This property enables O(log n) search, insert, and delete operations on average.

> 💡 **BST Property Visualization**
>
> ```

Valid BST:                  Invalid BST:
       8                          8
      / \                        / \
     3   10                     3   10
    / \    \                   / \    \
   1   6    14                1   9    14
      / \   /                    / \   /
     4   7 13                   4   7 13

Left of 8: {1,3,4,6,7}       Left of 8 contains 9 > 8!
All < 8 ✓                    BST property violated ✗

Inorder traversal of valid BST: 1, 3, 4, 6, 7, 8, 10, 13, 14
Notice: sorted order!
                
```

**Listing 6.8 — BST Validation**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def is_valid_bst(node, min_val=float('-inf'), max_val=float('inf')):
    """
    Check if tree is a valid BST.
    Each node must be within (min_val, max_val) range.
    """
    if node is None:
        return True
    
    # Check current node value is within bounds
    if node.data <= min_val or node.data >= max_val:
        return False
    
    # Left subtree: all values must be < current
    # Right subtree: all values must be > current
    return (is_valid_bst(node.left, min_val, node.data) and
            is_valid_bst(node.right, node.data, max_val))

# Valid BST
root1 = Node(8)
root1.left = Node(3)
root1.right = Node(10)
root1.left.left = Node(1)
root1.left.right = Node(6)

# Invalid BST (9 is in left subtree but > 8)
root2 = Node(8)
root2.left = Node(3)
root2.right = Node(10)
root2.left.left = Node(1)
root2.left.right = Node(9)  # Invalid!

print(f"Tree 1 is valid BST: {is_valid_bst(root1)}")
print(f"Tree 2 is valid BST: {is_valid_bst(root2)}")

***Figure 6.8:** BST validation tracks valid range for each node—don't just compare with parent!*

**Listing 6.9 — BST Search**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def search_recursive(node, target):
    """Search BST recursively - O(h) time."""
    if node is None:
        return None
    if target == node.data:
        return node
    elif target < node.data:
        return search_recursive(node.left, target)
    else:
        return search_recursive(node.right, target)

def search_iterative(root, target):
    """Search BST iteratively - O(h) time, O(1) space."""
    current = root
    while current:
        if target == current.data:
            return current
        elif target < current.data:
            current = current.left
        else:
            current = current.right
    return None

# Build BST:    8
#              / \
#             3   10
#            / \
#           1   6
root = Node(8)
root.left = Node(3)
root.right = Node(10)
root.left.left = Node(1)
root.left.right = Node(6)

# Search tests
for target in [6, 10, 5]:
    result = search_iterative(root, target)
    print(f"Search {target}: {'Found' if result else 'Not found'}")

***Figure 6.12:** BST search eliminates half the remaining nodes at each step—like binary search on arrays.*

**Listing 6.13 — BST Floor and Ceiling**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def floor(root, target):
    """Find largest value <= target."""
    result = None
    while root:
        if root.data == target:
            return root.data
        elif root.data < target:
            result = root.data  # Potential floor
            root = root.right
        else:
            root = root.left
    return result

def ceiling(root, target):
    """Find smallest value >= target."""
    result = None
    while root:
        if root.data == target:
            return root.data
        elif root.data > target:
            result = root.data  # Potential ceiling
            root = root.left
        else:
            root = root.right
    return result

# Build BST: [8, 3, 10, 1, 6, 14, 4, 7]
def insert(root, value):
    if root is None:
        return Node(value)
    if value < root.data:
        root.left = insert(root.left, value)
    else:
        root.right = insert(root.right, value)
    return root

root = None
for v in [8, 3, 10, 1, 6, 14, 4, 7]:
    root = insert(root, v)

print("BST contains: [1, 3, 4, 6, 7, 8, 10, 14]")
for target in [5, 7, 9, 15, 0]:
    print(f"  floor({target})={floor(root, target)}, ceiling({target})={ceiling(root, target)}")

***Figure 6.13:** Floor/ceiling operations find nearest values in O(h) time.*

---
## 6. BST Operations


### Insertion

**Listing 6.10 — BST Insertion**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def insert(root, value):
    """Insert value into BST, return root."""
    if root is None:
        return Node(value)
    
    if value < root.data:
        root.left = insert(root.left, value)
    else:
        root.right = insert(root.right, value)
    
    return root

def inorder(node):
    """Return inorder traversal."""
    if not node:
        return []
    return inorder(node.left) + [node.data] + inorder(node.right)

# Build BST by inserting values
root = None
values = [8, 3, 10, 1, 6, 14, 4, 7, 13]

print("Building BST:")
for v in values:
    root = insert(root, v)
    print(f"  Insert {v}: inorder = {inorder(root)}")

print(f"\nFinal BST inorder (sorted!): {inorder(root)}")

***Figure 6.10:** BST insertion always adds new nodes as leaves. Inorder traversal gives sorted order.*

### Finding Min and Max

**Listing 6.11 — BST Min/Max**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def find_min(node):
    """Minimum is the leftmost node."""
    if node is None:
        return None
    while node.left:
        node = node.left
    return node

def find_max(node):
    """Maximum is the rightmost node."""
    if node is None:
        return None
    while node.right:
        node = node.right
    return node

def find_successor(root, target):
    """Find inorder successor (next larger value)."""
    successor = None
    current = root
    
    while current:
        if target < current.data:
            successor = current  # Potential successor
            current = current.left
        else:
            current = current.right
    
    return successor

# Build BST
def insert(root, value):
    if root is None:
        return Node(value)
    if value < root.data:
        root.left = insert(root.left, value)
    else:
        root.right = insert(root.right, value)
    return root

root = None
for v in [8, 3, 10, 1, 6, 14, 4, 7, 13]:
    root = insert(root, v)

print(f"Minimum: {find_min(root).data}")
print(f"Maximum: {find_max(root).data}")

print("\nInorder successors:")
for val in [1, 6, 8, 13]:
    succ = find_successor(root, val)
    print(f"  Successor of {val}: {succ.data if succ else 'None'}")

***Figure 6.11:** In a BST, minimum is leftmost, maximum is rightmost. Both are O(h).*

### Deletion

**Listing 6.12 — BST Deletion**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def find_min(node):
    while node.left:
        node = node.left
    return node

def delete(root, value):
    """
    Delete value from BST. Three cases:
    1. Leaf: simply remove
    2. One child: replace with child
    3. Two children: replace with inorder successor
    """
    if root is None:
        return None
    
    # Find the node to delete
    if value < root.data:
        root.left = delete(root.left, value)
    elif value > root.data:
        root.right = delete(root.right, value)
    else:
        # Found the node to delete
        
        # Case 1 & 2: No child or one child
        if root.left is None:
            return root.right
        elif root.right is None:
            return root.left
        
        # Case 3: Two children
        # Find inorder successor (smallest in right subtree)
        successor = find_min(root.right)
        root.data = successor.data
        root.right = delete(root.right, successor.data)
    
    return root

def inorder(node):
    if not node:
        return []
    return inorder(node.left) + [node.data] + inorder(node.right)

def insert(root, value):
    if root is None:
        return Node(value)
    if value < root.data:
        root.left = insert(root.left, value)
    else:
        root.right = insert(root.right, value)
    return root

# Build BST
root = None
for v in [8, 3, 10, 1, 6, 14, 4, 7, 13]:
    root = insert(root, v)

print(f"Original: {inorder(root)}")

# Delete leaf
root = delete(root, 4)
print(f"Delete 4 (leaf): {inorder(root)}")

# Delete node with one child
root = delete(root, 14)
print(f"Delete 14 (one child): {inorder(root)}")

# Delete node with two children
root = delete(root, 3)
print(f"Delete 3 (two children): {inorder(root)}")

***Figure 6.12:** BST deletion handles three cases. Two-children case uses inorder successor to maintain BST property.*

---
## 7. BST Analysis


### Time Complexity

| Operation | Average Case | Worst Case |
| --- | --- | --- |
| Search | O(log n) | O(n) |
| Insert | O(log n) | O(n) |
| Delete | O(log n) | O(n) |
| Find Min/Max | O(log n) | O(n) |

The worst case O(n) occurs when the tree becomes a straight line (degenerate tree).

**Listing 6.13 — BST Degeneration**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def insert(root, value):
    if root is None:
        return Node(value)
    if value < root.data:
        root.left = insert(root.left, value)
    else:
        root.right = insert(root.right, value)
    return root

def height(node):
    if node is None:
        return -1
    return 1 + max(height(node.left), height(node.right))

def size(node):
    if node is None:
        return 0
    return 1 + size(node.left) + size(node.right)

# Balanced BST from random order
import random
balanced_values = list(range(1, 16))
random.shuffle(balanced_values)
balanced_root = None
for v in balanced_values:
    balanced_root = insert(balanced_root, v)

# Degenerate BST from sorted order
sorted_values = list(range(1, 16))
degenerate_root = None
for v in sorted_values:
    degenerate_root = insert(degenerate_root, v)

print("Inserting values 1-15:")
print(f"\nBalanced (random order):")
print(f"  Height: {height(balanced_root)}")
print(f"  Ideal height: ~{int(3.9)}")  # log2(15) ≈ 3.9

print(f"\nDegenerate (sorted order):")
print(f"  Height: {height(degenerate_root)}")
print(f"  This is essentially a linked list!")

print("\nSolution: Use self-balancing trees (AVL, Red-Black)")

***Figure 6.16:** Inserting sorted data creates a degenerate BST with O(n) operations. Self-balancing trees solve this.*

**Listing 6.17 — BST vs Balanced Tree Performance**

In [ ]:
import time
import sys
sys.setrecursionlimit(2000)

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def insert(root, value):
    if root is None:
        return Node(value)
    if value < root.data:
        root.left = insert(root.left, value)
    else:
        root.right = insert(root.right, value)
    return root

def search(root, target):
    while root:
        if target == root.data:
            return True
        elif target < root.data:
            root = root.left
        else:
            root = root.right
    return False

n = 1000

# Degenerate BST (sorted insert)
sorted_root = None
start = time.perf_counter()
for i in range(n):
    sorted_root = insert(sorted_root, i)
sorted_insert_time = time.perf_counter() - start

# Search in degenerate BST
start = time.perf_counter()
for i in range(0, n, 10):
    search(sorted_root, i)
sorted_search_time = time.perf_counter() - start

# Random order BST (more balanced)
import random
values = list(range(n))
random.shuffle(values)
random_root = None
start = time.perf_counter()
for v in values:
    random_root = insert(random_root, v)
random_insert_time = time.perf_counter() - start

# Search in random BST
start = time.perf_counter()
for i in range(0, n, 10):
    search(random_root, i)
random_search_time = time.perf_counter() - start

print(f"Insert {n} elements:")
print(f"  Sorted order (degenerate): {sorted_insert_time*1000:.2f} ms")
print(f"  Random order (balanced):   {random_insert_time*1000:.2f} ms")
print(f"\nSearch 100 elements:")
print(f"  Degenerate BST: {sorted_search_time*1000:.2f} ms")
print(f"  Balanced BST:   {random_search_time*1000:.2f} ms")

***Figure 6.17:** Degenerate trees have dramatically worse performance than balanced trees.*

### AVL Trees: Self-Balancing BSTs

A BST degenerates to O(n) when nodes are inserted in sorted order. **AVL trees** (named after Adelson-Velsky and Landis) prevent this by maintaining a balance invariant: for every node, the heights of its left and right subtrees differ by at most 1. After every insertion or deletion, the tree rebalances itself using *rotations*.

> 📖 **AVL Balance Invariant:** For every node in an AVL tree: |height(left subtree) − height(right subtree)| ≤ 1. This guarantees the tree height is always O(log n), so all operations (search, insert, delete) are O(log n) worst case.

**Listing 6.18 — AVL Tree with Rotations**

In [ ]:
class AVLNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1  # New nodes are leaves with height 1

class AVLTree:
    def _height(self, node):
        return node.height if node else 0

    def _balance_factor(self, node):
        return self._height(node.left) - self._height(node.right)

    def _update_height(self, node):
        node.height = 1 + max(self._height(node.left), self._height(node.right))

    # ===== THE FOUR ROTATIONS =====

    def _rotate_right(self, y):
        """Right rotation (for Left-Left case):
              y              x
             / \            / \
            x   C   =>    A   y
           / \                / \
          A   B              B   C
        """
        x = y.left
        B = x.right
        x.right = y
        y.left = B
        self._update_height(y)
        self._update_height(x)
        return x  # x is the new root

    def _rotate_left(self, x):
        """Left rotation (for Right-Right case):
          x                y
         / \              / \
        A   y     =>    x   C
           / \         / \
          B   C       A   B
        """
        y = x.right
        B = y.left
        y.left = x
        x.right = B
        self._update_height(x)
        self._update_height(y)
        return y  # y is the new root

    def _rebalance(self, node):
        """Check balance and apply rotations if needed."""
        self._update_height(node)
        bf = self._balance_factor(node)

        # Left-heavy (bf > 1)
        if bf > 1:
            if self._balance_factor(node.left) < 0:
                # Left-Right case: left rotate child first
                node.left = self._rotate_left(node.left)
            return self._rotate_right(node)  # Left-Left case

        # Right-heavy (bf < -1)
        if bf < -1:
            if self._balance_factor(node.right) > 0:
                # Right-Left case: right rotate child first
                node.right = self._rotate_right(node.right)
            return self._rotate_left(node)  # Right-Right case

        return node  # Already balanced

    def insert(self, root, key):
        """Insert key and rebalance. Returns new root."""
        if not root:
            return AVLNode(key)
        if key < root.key:
            root.left = self.insert(root.left, key)
        elif key > root.key:
            root.right = self.insert(root.right, key)
        else:
            return root  # Duplicate keys not allowed
        return self._rebalance(root)

    def inorder(self, node, result=None):
        if result is None:
            result = []
        if node:
            self.inorder(node.left, result)
            result.append(f"{node.key}(h={node.height})")
            self.inorder(node.right, result)
        return result

# === Demo: Insert sorted data (worst case for plain BST) ===
avl = AVLTree()
root = None

print("Inserting 1, 2, 3, 4, 5, 6, 7 (sorted — would create degenerate BST):")
for key in [1, 2, 3, 4, 5, 6, 7]:
    root = avl.insert(root, key)
    print(f"  Insert {key} → root={root.key}, height={root.height}, "
          f"balance={avl._balance_factor(root)}")

print(f"\nInorder: {avl.inorder(root)}")
print(f"Tree height: {root.height} (plain BST would be 7!)")
print(f"Root: {root.key} (balanced around middle value)")

***Figure 6.18:** AVL tree maintains O(log n) height even with sorted insertions. The four rotation cases (LL, RR, LR, RL) cover all possible imbalances.*

---
## 8. Tree Applications


### Application 1: Expression Trees

**Listing 6.14 — Expression Tree**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def evaluate(node):
    """Evaluate expression tree."""
    if node is None:
        return 0
    
    # Leaf node = operand
    if node.left is None and node.right is None:
        return int(node.data)
    
    # Internal node = operator
    left_val = evaluate(node.left)
    right_val = evaluate(node.right)
    
    if node.data == '+':
        return left_val + right_val
    elif node.data == '-':
        return left_val - right_val
    elif node.data == '*':
        return left_val * right_val
    elif node.data == '/':
        return left_val // right_val

def infix(node):
    """Convert to infix notation with parentheses."""
    if node.left is None:
        return node.data
    return f"({infix(node.left)} {node.data} {infix(node.right)})"

# Build expression tree for: (3 + 4) * (5 - 2)
#           *
#          / \
#         +   -
#        / \ / \
#       3  4 5  2
root = Node('*')
root.left = Node('+')
root.right = Node('-')
root.left.left = Node('3')
root.left.right = Node('4')
root.right.left = Node('5')
root.right.right = Node('2')

print(f"Expression: {infix(root)}")
print(f"Result: {evaluate(root)}")

***Figure 6.14:** Expression trees represent mathematical expressions. Postorder traversal evaluates them.*

### Application 2: File System Representation

**Listing 6.15 — File System Tree**

In [ ]:
class FileNode:
    def __init__(self, name, is_dir=False, size=0):
        self.name = name
        self.is_dir = is_dir
        self.size = size  # For files
        self.children = []  # For directories
    
    def add_child(self, child):
        if self.is_dir:
            self.children.append(child)

def print_tree(node, prefix=""):
    """Print file system tree."""
    print(f"{prefix}{node.name}")
    for i, child in enumerate(node.children):
        is_last = (i == len(node.children) - 1)
        new_prefix = prefix + ("    " if is_last else "│   ")
        connector = "└── " if is_last else "├── "
        print(f"{prefix}{connector}{child.name}" + 
              (f" ({child.size} bytes)" if not child.is_dir else ""))
        if child.is_dir:
            print_tree(child, new_prefix)

def total_size(node):
    """Calculate total size including subdirectories."""
    if not node.is_dir:
        return node.size
    return sum(total_size(child) for child in node.children)

# Build file system
root = FileNode("/", is_dir=True)
home = FileNode("home", is_dir=True)
docs = FileNode("documents", is_dir=True)
file1 = FileNode("readme.txt", size=1024)
file2 = FileNode("data.csv", size=5120)
file3 = FileNode("photo.jpg", size=2048)

root.add_child(home)
root.add_child(FileNode("etc", is_dir=True))
home.add_child(docs)
home.add_child(file3)
docs.add_child(file1)
docs.add_child(file2)

print("File System Structure:")
print_tree(root)
print(f"\nTotal size of /home: {total_size(home)} bytes")

***Figure 6.15:** File systems are naturally represented as trees. Directories are internal nodes, files are leaves.*

### Application 3: Autocomplete with Trie

**Listing 6.16 — Trie for Autocomplete**

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {}  # char -> TrieNode
        self.is_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_word = True
    
    def search(self, word):
        node = self._find_node(word)
        return node is not None and node.is_word
    
    def starts_with(self, prefix):
        return self._find_node(prefix) is not None
    
    def _find_node(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return None
            node = node.children[char]
        return node
    
    def autocomplete(self, prefix):
        """Return all words starting with prefix."""
        node = self._find_node(prefix)
        if not node:
            return []
        
        results = []
        self._collect_words(node, prefix, results)
        return results
    
    def _collect_words(self, node, prefix, results):
        if node.is_word:
            results.append(prefix)
        for char, child in node.children.items():
            self._collect_words(child, prefix + char, results)

# Test autocomplete
trie = Trie()
words = ["apple", "app", "application", "apply", "apt", "banana", "band"]
for word in words:
    trie.insert(word)

print("Autocomplete results:")
for prefix in ["app", "ap", "ban", "xyz"]:
    results = trie.autocomplete(prefix)
    print(f"  '{prefix}': {results}")

***Figure 6.19:** Tries enable O(k) search where k is the key length. Perfect for autocomplete and spell-checking.*

### Application 4: Binary Tree from Arrays

**Listing 6.20 — Build Tree from Inorder and Preorder**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def build_tree(preorder, inorder):
    """
    Reconstruct binary tree from preorder and inorder traversals.
    Preorder: Root comes first
    Inorder: Separates left and right subtrees
    """
    if not preorder or not inorder:
        return None
    
    # Root is first element of preorder
    root_val = preorder[0]
    root = Node(root_val)
    
    # Find root position in inorder
    root_idx = inorder.index(root_val)
    
    # Build subtrees
    root.left = build_tree(preorder[1:root_idx+1], inorder[:root_idx])
    root.right = build_tree(preorder[root_idx+1:], inorder[root_idx+1:])
    
    return root

def postorder(node):
    if not node:
        return []
    return postorder(node.left) + postorder(node.right) + [node.data]

# Test
preorder = [3, 9, 20, 15, 7]
inorder = [9, 3, 15, 20, 7]

root = build_tree(preorder, inorder)
print(f"Preorder: {preorder}")
print(f"Inorder: {inorder}")
print(f"Built tree postorder: {postorder(root)}")

***Figure 6.20:** Given preorder and inorder, the tree can be uniquely reconstructed.*

### Application 5: Right Side View

**Listing 6.21 — Binary Tree Right Side View**

In [ ]:
from collections import deque

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def right_side_view(root):
    """Return values visible from right side of tree."""
    if not root:
        return []
    
    result = []
    queue = deque([root])
    
    while queue:
        level_size = len(queue)
        
        for i in range(level_size):
            node = queue.popleft()
            
            # Last node in this level is visible from right
            if i == level_size - 1:
                result.append(node.data)
            
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
    
    return result

# Build tree:     1
#               / \
#              2   3
#               \   \
#                5   4
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.right = Node(5)
root.right.right = Node(4)

print(f"Right side view: {right_side_view(root)}")
# From right: see 1, then 3, then 4

***Figure 6.21:** Right side view shows the rightmost node at each level.*

### Application 6: Flatten Tree to Linked List

**Listing 6.22 — Flatten Binary Tree**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def flatten(root):
    """
    Flatten tree to linked list in-place (preorder).
    All nodes end up as right children.
    """
    if not root:
        return
    
    # Flatten left and right subtrees
    flatten(root.left)
    flatten(root.right)
    
    # Store right subtree
    right_subtree = root.right
    
    # Move left subtree to right
    root.right = root.left
    root.left = None
    
    # Find end of new right subtree and attach old right
    current = root
    while current.right:
        current = current.right
    current.right = right_subtree

def print_list(node):
    """Print flattened tree as list."""
    result = []
    while node:
        result.append(node.data)
        node = node.right
    return result

# Build tree:     1
#               / \
#              2   5
#             / \   \
#            3   4   6
root = Node(1)
root.left = Node(2)
root.right = Node(5)
root.left.left = Node(3)
root.left.right = Node(4)
root.right.right = Node(6)

flatten(root)
print(f"Flattened: {print_list(root)}")
# Should be: 1 -> 2 -> 3 -> 4 -> 5 -> 6

***Figure 6.22:** Flattening converts tree to linked list following preorder sequence.*

---
## 9. Common Patterns


### Pattern 1: Lowest Common Ancestor

**Listing 6.17 — LCA in Binary Tree**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def lca(root, p, q):
    """
    Find lowest common ancestor of nodes p and q.
    LCA is the deepest node that has both p and q as descendants.
    """
    if root is None:
        return None
    
    # If current node is p or q, it's the LCA
    if root.data == p or root.data == q:
        return root
    
    # Search in left and right subtrees
    left_lca = lca(root.left, p, q)
    right_lca = lca(root.right, p, q)
    
    # If both found something, current node is LCA
    if left_lca and right_lca:
        return root
    
    # Otherwise return the non-null result
    return left_lca if left_lca else right_lca

def lca_bst(root, p, q):
    """LCA in BST is simpler - use BST property."""
    while root:
        if p < root.data and q < root.data:
            root = root.left
        elif p > root.data and q > root.data:
            root = root.right
        else:
            return root
    return None

# Build tree:    3
#              / \
#             5   1
#            / \
#           6   2
root = Node(3)
root.left = Node(5)
root.right = Node(1)
root.left.left = Node(6)
root.left.right = Node(2)

print("LCA tests (general binary tree):")
print(f"  LCA(5, 1) = {lca(root, 5, 1).data}")  # 3
print(f"  LCA(5, 6) = {lca(root, 5, 6).data}")  # 5
print(f"  LCA(6, 2) = {lca(root, 6, 2).data}")  # 5

***Figure 6.17:** LCA is found where paths to p and q diverge. BST version exploits ordering.*

### Pattern 2: Path Sum

**Listing 6.18 — Path Sum Problems**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def has_path_sum(root, target_sum):
    """Check if any root-to-leaf path equals target_sum."""
    if root is None:
        return False
    
    # Leaf node: check if remaining sum equals node value
    if root.left is None and root.right is None:
        return root.data == target_sum
    
    # Recurse with reduced target
    remaining = target_sum - root.data
    return (has_path_sum(root.left, remaining) or
            has_path_sum(root.right, remaining))

def all_paths(root, target_sum):
    """Find all root-to-leaf paths that sum to target."""
    results = []
    
    def dfs(node, remaining, path):
        if node is None:
            return
        
        path.append(node.data)
        
        # Leaf node
        if node.left is None and node.right is None:
            if remaining == node.data:
                results.append(path.copy())
        else:
            dfs(node.left, remaining - node.data, path)
            dfs(node.right, remaining - node.data, path)
        
        path.pop()  # Backtrack
    
    dfs(root, target_sum, [])
    return results

# Build tree:     5
#               / \
#              4   8
#             /   / \
#            11  13  4
#           /  \    / \
#          7    2  5   1
root = Node(5)
root.left = Node(4)
root.right = Node(8)
root.left.left = Node(11)
root.right.left = Node(13)
root.right.right = Node(4)
root.left.left.left = Node(7)
root.left.left.right = Node(2)
root.right.right.left = Node(5)
root.right.right.right = Node(1)

target = 22
print(f"Has path sum {target}: {has_path_sum(root, target)}")
print(f"All paths with sum {target}: {all_paths(root, target)}")

***Figure 6.18:** Path sum uses DFS with backtracking. Track the current path and sum while exploring.*

### Pattern 3: Serialize and Deserialize

**Listing 6.19 — Tree Serialization**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def serialize(root):
    """Convert tree to string using preorder with null markers."""
    def helper(node):
        if node is None:
            return ["#"]
        return [str(node.data)] + helper(node.left) + helper(node.right)
    
    return ",".join(helper(root))

def deserialize(data):
    """Reconstruct tree from serialized string."""
    values = iter(data.split(","))
    
    def helper():
        val = next(values)
        if val == "#":
            return None
        node = Node(int(val))
        node.left = helper()
        node.right = helper()
        return node
    
    return helper()

def inorder(node):
    if not node:
        return []
    return inorder(node.left) + [node.data] + inorder(node.right)

# Build tree
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.right.left = Node(4)
root.right.right = Node(5)

print(f"Original tree (inorder): {inorder(root)}")

# Serialize
serialized = serialize(root)
print(f"Serialized: {serialized}")

# Deserialize
restored = deserialize(serialized)
print(f"Restored tree (inorder): {inorder(restored)}")

***Figure 6.19:** Serialization converts a tree to a string. Preorder with null markers enables unique reconstruction.*

### Pattern 4: Tree Diameter

**Listing 6.20 — Tree Diameter**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def diameter(root):
    """
    Diameter = longest path between any two nodes.
    Path may or may not pass through root.
    """
    max_diameter = [0]  # Use list to allow modification in nested function
    
    def height(node):
        if node is None:
            return 0
        
        left_height = height(node.left)
        right_height = height(node.right)
        
        # Update diameter: path through this node
        max_diameter[0] = max(max_diameter[0], 
                              left_height + right_height)
        
        return 1 + max(left_height, right_height)
    
    height(root)
    return max_diameter[0]

# Build tree:     1
#               / \
#              2   3
#             / \
#            4   5
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)

print(f"Tree diameter: {diameter(root)}")
# Diameter is path 4 -> 2 -> 1 -> 3 = 3 edges
# Or 4 -> 2 -> 5 = 2 edges, or 5 -> 2 -> 1 -> 3 = 3 edges

***Figure 6.20:** Diameter is computed during height calculation. At each node, check if path through it is longest.*

### Pattern 5: Check Balanced Tree

**Listing 6.21 — Balanced Tree Check**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def is_balanced(root):
    """
    Check if tree is height-balanced.
    Balanced = height difference of subtrees <= 1 at every node.
    """
    def check(node):
        if node is None:
            return 0  # Height of empty tree
        
        left_height = check(node.left)
        if left_height == -1:
            return -1  # Left subtree unbalanced
        
        right_height = check(node.right)
        if right_height == -1:
            return -1  # Right subtree unbalanced
        
        if abs(left_height - right_height) > 1:
            return -1  # Current node unbalanced
        
        return 1 + max(left_height, right_height)
    
    return check(root) != -1

# Balanced tree
balanced = Node(1)
balanced.left = Node(2)
balanced.right = Node(3)
balanced.left.left = Node(4)

# Unbalanced tree
unbalanced = Node(1)
unbalanced.left = Node(2)
unbalanced.left.left = Node(3)
unbalanced.left.left.left = Node(4)

print(f"Balanced tree: {is_balanced(balanced)}")
print(f"Unbalanced tree: {is_balanced(unbalanced)}")

***Figure 6.28:** Efficient balance check computes height while checking balance, returning -1 early if unbalanced.*

### Pattern 6: Zigzag Level Order

**Listing 6.29 — Zigzag Traversal**

In [ ]:
from collections import deque

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def zigzag_level_order(root):
    """Traverse tree level by level, alternating direction."""
    if not root:
        return []
    
    result = []
    queue = deque([root])
    left_to_right = True
    
    while queue:
        level_size = len(queue)
        level = deque()
        
        for _ in range(level_size):
            node = queue.popleft()
            
            if left_to_right:
                level.append(node.data)
            else:
                level.appendleft(node.data)
            
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
        
        result.append(list(level))
        left_to_right = not left_to_right
    
    return result

# Build tree:     3
#               / \
#              9  20
#                / \
#               15  7
root = Node(3)
root.left = Node(9)
root.right = Node(20)
root.right.left = Node(15)
root.right.right = Node(7)

print(f"Zigzag level order: {zigzag_level_order(root)}")
# [[3], [20, 9], [15, 7]]

***Figure 6.29:** Zigzag traversal alternates between left-to-right and right-to-left at each level.*

### Pattern 7: Count Complete Tree Nodes

**Listing 6.30 — Count Nodes in Complete Tree**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def count_nodes(root):
    """
    Count nodes in complete binary tree in O(log^2 n) time.
    Uses the property that one subtree is always perfect.
    """
    if not root:
        return 0
    
    left_height = get_height(root.left)
    right_height = get_height(root.right)
    
    if left_height == right_height:
        # Left subtree is perfect
        # Nodes = 2^left_height (left subtree + root) + count right
        return (1 << left_height) + count_nodes(root.right)
    else:
        # Right subtree is perfect (one level less)
        # Nodes = 2^right_height (right subtree + root) + count left
        return (1 << right_height) + count_nodes(root.left)

def get_height(node):
    """Height by going left (complete tree property)."""
    height = 0
    while node:
        height += 1
        node = node.left
    return height

# Build complete tree with 6 nodes
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)
root.right.left = Node(6)

print(f"Node count: {count_nodes(root)}")
print("This is O(log^2 n), not O(n)!")

***Figure 6.30:** Complete tree node counting exploits the structure for O(log² n) complexity.*

- **Trees** are hierarchical structures with nodes and edges
- **Binary trees** have at most 2 children per node
- **Traversals:** preorder (root first), inorder (sorted for BST), postorder (children first), level-order (BFS)
- **BST property:** left < root < right enables O(log n) operations
- **BST worst case:** O(n) when degenerate—use balanced trees
- **Common patterns:** recursion, DFS/BFS, path tracking, height calculation

---
## 10. Common Pitfalls


### Pitfall 1: Wrong BST Validation

**Listing 6.22 — BST Validation Mistakes**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

# WRONG: Only checks immediate children
def is_bst_wrong(node):
    if node is None:
        return True
    if node.left and node.left.data >= node.data:
        return False
    if node.right and node.right.data <= node.data:
        return False
    return is_bst_wrong(node.left) and is_bst_wrong(node.right)

# CORRECT: Tracks valid range
def is_bst_correct(node, min_val=float('-inf'), max_val=float('inf')):
    if node is None:
        return True
    if node.data <= min_val or node.data >= max_val:
        return False
    return (is_bst_correct(node.left, min_val, node.data) and
            is_bst_correct(node.right, node.data, max_val))

# This tree passes wrong validation but is NOT a valid BST!
#       10
#      /  \
#     5    15
#    / \
#   1   12  <- 12 > 10, violates BST!
root = Node(10)
root.left = Node(5)
root.right = Node(15)
root.left.left = Node(1)
root.left.right = Node(12)  # Invalid!

print(f"Wrong validation: {is_bst_wrong(root)}")  # True (incorrect!)
print(f"Correct validation: {is_bst_correct(root)}")  # False

***Figure 6.22:** BST validation must check entire valid range, not just immediate parent.*

### Pitfall 2: Forgetting Base Cases

**Listing 6.23 — Proper Base Cases**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

# BAD: Missing None check causes AttributeError
def count_nodes_bad(node):
    return 1 + count_nodes_bad(node.left) + count_nodes_bad(node.right)

# GOOD: Always check for None first
def count_nodes_good(node):
    if node is None:  # Base case!
        return 0
    return 1 + count_nodes_good(node.left) + count_nodes_good(node.right)

# Test
root = Node(1)
root.left = Node(2)

print(f"Good function: {count_nodes_good(root)}")

try:
    count_nodes_bad(root)
except AttributeError as e:
    print(f"Bad function error: {e}")

print("\nAlways handle None at the start of recursive tree functions!")

***Figure 6.23:** Always check for None at the start of recursive tree functions.*

### Pitfall 3: Modifying Tree During Traversal

**Listing 6.24 — Safe Tree Modification**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def collect_nodes_to_delete(root, target):
    """Collect nodes to delete, don't delete during traversal."""
    to_delete = []
    
    def traverse(node):
        if node is None:
            return
        if node.data == target:
            to_delete.append(node)
        traverse(node.left)
        traverse(node.right)
    
    traverse(root)
    return to_delete

def inorder(node):
    if not node:
        return []
    return inorder(node.left) + [node.data] + inorder(node.right)

# Build tree
root = Node(1)
root.left = Node(2)
root.right = Node(3)

print(f"Tree: {inorder(root)}")
print("\nTip: Never modify a tree while traversing it.")
print("Instead, collect what to modify, then modify separately.")

***Figure 6.24:** Collect nodes to modify during traversal, then modify in a separate pass.*

---
# 📝 Exercises


### Exercise 1: Maximum Depth of Binary Tree  (⭐ Easy)

Write a function that returns the maximum depth (height) of a binary tree. An empty tree has depth 0, a single node has depth 1.

**Expected:** (Expected: Tree with 3 levels returns 3)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Base Case:**

                        If node is `None`, return 0 (empty tree has depth 0).
- **Hint 2 - Recursive Calls:**

                        Get the max depth of left subtree and right subtree recursively.
- **Hint 3 - Combine Results:**

                        Return `1 + max(left_depth, right_depth)`
</details>

In [ ]:
# ✏️ [EX1]
# Maximum Depth of Binary Tree

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def max_depth(root):
    # Your code here
    pass

# Test your implementation (uncomment)
# root = Node(3)
# root.left = Node(9)
# root.right = Node(20)
# root.right.left = Node(15)
# root.right.right = Node(7)
# print(f"Max depth: {max_depth(root)}")  # Expected: 3

### Exercise 2: Invert Binary Tree  (⭐ Easy)

Write a function that inverts (mirrors) a binary tree. Swap left and right children at every node.

**Expected:** (Expected: Left and right subtrees swapped at every level)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Base Case:**

                        If node is `None`, return `None`.
- **Hint 2 - Swap Children:**

                        Swap `node.left` and `node.right`.
- **Hint 3 - Recurse:**

                        Recursively invert both subtrees (order doesn't matter).
</details>

In [ ]:
# ✏️ [EX2]
# Invert Binary Tree - Mirror the tree

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def invert_tree(root):
    # Your code here
    pass

# Test your implementation (uncomment)
# root = Node(4)
# root.left = Node(2)
# root.right = Node(7)
# root.left.left = Node(1)
# root.left.right = Node(3)
# invert_tree(root)
# # After invert: 4 -> [7, 2], 7 -> [], 2 -> [3, 1]

### Exercise 3: Kth Smallest in BST  (⭐⭐ Medium)

Given a BST, find the kth smallest element (1-indexed).

**Expected:** (Expected: Inorder traversal gives sorted order)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - BST Property:**

                        Inorder traversal of BST visits nodes in sorted (ascending) order.
- **Hint 2 - Count Nodes:**

                        Count nodes as you traverse inorder, stop when count equals k.
- **Hint 3 - Iterative Approach:**

                        Use a stack for iterative inorder traversal to stop early.
</details>

In [ ]:
# ✏️ [EX3]
# Kth Smallest in BST - Use inorder traversal

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def kth_smallest(root, k):
    # Your code here
    pass

# Test your implementation (uncomment)
# root = Node(5)
# root.left = Node(3)
# root.right = Node(6)
# root.left.left = Node(2)
# root.left.right = Node(4)
# root.left.left.left = Node(1)
# print(f"1st smallest: {kth_smallest(root, 1)}")  # 1
# print(f"3rd smallest: {kth_smallest(root, 3)}")  # 3

### Exercise 4: Symmetric Tree  (⭐⭐ Medium)

Check if a binary tree is symmetric (mirror of itself around the center).

**Expected:** (Expected: Compare left subtree with right subtree recursively)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Mirror Property:**

                        Tree is symmetric if left subtree mirrors right subtree.
- **Hint 2 - Compare Mirrored Positions:**

                        Compare `left.left` with `right.right`, and `left.right` with `right.left`.
- **Hint 3 - Base Cases:**

                        Both None → True, one None → False, values differ → False.
</details>

In [ ]:
# ✏️ [EX4]
# Symmetric Tree - Check if tree is mirror of itself

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def is_symmetric(root):
    # Your code here
    pass

# Test your implementation (uncomment)
# Symmetric:    1
#              / \
#             2   2
#            / \ / \
#           3  4 4  3
# symmetric = Node(1)
# symmetric.left = Node(2)
# symmetric.right = Node(2)
# symmetric.left.left = Node(3)
# symmetric.left.right = Node(4)
# symmetric.right.left = Node(4)
# symmetric.right.right = Node(3)
# print(f"Is symmetric: {is_symmetric(symmetric)}")  # True

### Exercise 5: Construct BST from Preorder  (⭐⭐⭐ Hard)

Given a preorder traversal of a BST, construct the original BST.

**Expected:** (Expected: First element is root, partition by BST property)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Root First:**

                        First element of preorder is always the root.
- **Hint 2 - Partition:**

                        Elements smaller than root go to left subtree, larger go to right.
- **Hint 3 - Recursive Construction:**

                        Find split point, recursively build left and right subtrees.
</details>

In [ ]:
# ✏️ [EX5]
# Construct BST from Preorder Traversal

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def bst_from_preorder(preorder):
    # Your code here
    pass

def inorder(node):
    if not node:
        return []
    return inorder(node.left) + [node.data] + inorder(node.right)

# Test your implementation (uncomment)
# preorder = [8, 5, 1, 7, 10, 12]
# root = bst_from_preorder(preorder)
# print(f"Inorder: {inorder(root)}")  # [1, 5, 7, 8, 10, 12]

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Save this notebook** (Ctrl+S)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,10}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-10 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you SAVED the notebook first! (Ctrl+S)

import json, re, os, urllib.request

WEEK = "Week_06"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ── Check info was filled in ──
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Run the cell above first to set your info!")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Run the cell above first — some fields are empty.")

# ── Find this notebook file ──
_nb_path = None

# VS Code
try:
    _nb_path = __vsc_ipynb_file__
except NameError:
    pass

# Colab
if not _nb_path:
    try:
        import google.colab
        _candidates = [f for f in os.listdir(".") if f.endswith(".ipynb") and WEEK in f]
        if _candidates:
            _nb_path = _candidates[0]
    except ImportError:
        pass

# Fallback: search current dir
if not _nb_path:
    _candidates = [f for f in os.listdir(".") if f.endswith(".ipynb") and WEEK in f]
    if len(_candidates) == 1:
        _nb_path = _candidates[0]

if not _nb_path or not os.path.exists(str(_nb_path)):
    print("⚠️  Could not auto-detect notebook file.")
    print("   Available .ipynb files:", [f for f in os.listdir(".") if f.endswith(".ipynb")])
    raise SystemExit("Please make sure the notebook is saved and in the current directory.")

print(f"📖 Reading {os.path.basename(str(_nb_path))}...")

with open(str(_nb_path), "r", encoding="utf-8") as _f:
    _nb = json.load(_f)

# ── Extract exercise answers ──
_answers = {}
for _cell in _nb["cells"]:
    if _cell["cell_type"] != "code":
        continue
    _src = "".join(_cell["source"]) if isinstance(_cell["source"], list) else _cell["source"]
    _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
    if _m:
        _ex_id = "ex" + _m.group(1)
        _lines = _src.split("\n")
        _clean = "\n".join(_lines[1:]).strip()
        _answers[_ex_id] = {
            "code": _clean,
            "modified": len(_clean) > 5
        }

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys()))}")

if not _answers:
    print("\n⚠️  No exercise answers found!")
    print("Make sure exercise cells still have the # ✏️ [EX...] tag.")
    raise SystemExit()

# ── Send ──
_data = json.dumps({
    "week": WEEK,
    "studentId": _sid,
    "studentName": _sname,
    "studentEmail": _semail,
    "classCode": _scode,
    "source": "dsa-notebook",
    "timeOnPage": 0,
    "answers": _answers
}).encode("utf-8")

print("📡 Submitting...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get("success"):
        print(f"\n✅ {_result['message']}")
        print("📧 Check your email for confirmation.")
    else:
        print(f"\n❌ {_result.get('message', 'Submission failed')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  Request sent — check your email for confirmation.")
    print(f"(If no email arrives, try again or contact your instructor)")
